# Generate Choice file for all participants (pet experiment)


In [17]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame as DF
# Useful to quickly extract csv files
import glob
import os

In [18]:
# Extract Rate data

In [19]:
# Load data all participants

rateNameFrames = glob.glob('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/*_RateData_PFT.csv')
choiceNameFrames = glob.glob('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/*_ChoiceData_PFT_MRI.csv')

rateNameFrames.sort()
choiceNameFrames.sort()

In [20]:
rateNameFrames

['/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/S1_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/S2_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/S3_RateData_PFT.csv']

In [21]:
choiceNameFrames

['/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/pilot1_ChoiceData_PFT_MRI.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/pilot2_ChoiceData_PFT_MRI.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/pilot3_ChoiceData_PFT_MRI.csv']

## Join rating in this case

In [22]:
partic_num=len(rateNameFrames) #number of participants

choiceFrameAll = pd.DataFrame( columns = ['trial','leftId','rightId','frame','endTime','choice','choiceTime','LRat1','RRat1','LRat2','RRat2','LVar1','RVar1','LVar2','RVar2','Part'])
idFrameAll = pd.DataFrame( columns = ['nameId','rat1','rat2','var1','var2','Part'])

for j in range (partic_num):
    rateFrame = pd.read_csv(rateNameFrames[j])
    choiceFrames = pd.read_csv(choiceNameFrames[j])
    
    if 'ratL' in choiceFrames.columns:
        if 'ratR' in choiceFrames.columns:
            choiceFrames = choiceFrames.drop(['ratL', 'ratR'], axis=1)

    
    idNames = list(rateFrame.picId.unique())
    IdRatings1 = [None]*len(idNames)
    IdRatings2 = [None]*len(idNames)
    IdRatingsVar1 = [None]*len(idNames)
    IdRatingsVar2 = [None]*len(idNames)

    
    #extract average rating for like, dislike
    for i in range(len(idNames)):
        IdRatings1[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 1)].rating.mean()
        IdRatings2[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 2)].rating.mean()
        
        IdRatingsVar1[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 1)].rating.std()
        IdRatingsVar2[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 2)].rating.std()

    rateValues = pd.DataFrame( {'nameId': idNames,'rat1': IdRatings1,'rat2': IdRatings2,'var1': IdRatingsVar1,'var2': IdRatingsVar2,})
    rateValues['Part'] = [j]*len(rateValues)    
    
    # save information about celebs and ratings
    
    idFrameAll = pd.concat([idFrameAll,rateValues],ignore_index=True)

    # separate ratings and rating-variability for each frame
    ChoiceLRat1 = [None]*len(choiceFrames)
    ChoiceRRat1 = [None]*len(choiceFrames)
    ChoiceLRat2 = [None]*len(choiceFrames)
    ChoiceRRat2 = [None]*len(choiceFrames)

    ChoiceLVar1 = [None]*len(choiceFrames)
    ChoiceRVar1 = [None]*len(choiceFrames)
    ChoiceLVar2 = [None]*len(choiceFrames)
    ChoiceRVar2 = [None]*len(choiceFrames)
    
    for i in range(len(choiceFrames)):
        
        choiceL = choiceFrames.iloc[i].leftId
        choiceR = choiceFrames.iloc[i].rightId
        
        if any((rateValues['nameId'] == choiceL)) and any((rateValues['nameId'] == choiceR)):
        
            ChoiceLRat1[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].rat1.values[0]
            ChoiceRRat1[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].rat1.values[0]
            ChoiceLRat2[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].rat2.values[0]
            ChoiceRRat2[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].rat2.values[0]
            
            ChoiceLVar1[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].var1.values[0]
            ChoiceRVar1[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].var1.values[0]
            ChoiceLVar2[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].var2.values[0]
            ChoiceRVar2[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].var2.values[0]

    choiceFrames['LRat1'] = ChoiceLRat1
    choiceFrames['RRat1'] = ChoiceRRat1
    choiceFrames['LRat2'] = ChoiceLRat2
    choiceFrames['RRat2'] = ChoiceRRat2 
    
    choiceFrames['LVar1'] = ChoiceLVar1
    choiceFrames['RVar1'] = ChoiceRVar1
    choiceFrames['LVar2'] = ChoiceLVar2
    choiceFrames['RVar2'] = ChoiceRVar2
     
    choiceFrames['Part'] = [j]*len(choiceFrames)    

    choiceFrameAll = pd.concat([choiceFrameAll,choiceFrames],ignore_index=True)
    

/Users/pradyumna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [23]:
choiceFrameAll.frame

0      1.0
1      1.0
2      2.0
3      1.0
4      1.0
5      2.0
6      2.0
7      1.0
8      2.0
9      2.0
10     1.0
11     1.0
12     2.0
13     1.0
14     2.0
15     2.0
16     2.0
17     1.0
18     2.0
19     2.0
20     1.0
21     1.0
22     2.0
23     2.0
24     2.0
25     1.0
26     1.0
27     1.0
28     1.0
29     1.0
      ... 
162    2.0
163    2.0
164    1.0
165    2.0
166    2.0
167    1.0
168    1.0
169    2.0
170    1.0
171    1.0
172    2.0
173    2.0
174    1.0
175    2.0
176    2.0
177    1.0
178    2.0
179    1.0
180    1.0
181    1.0
182    1.0
183    2.0
184    2.0
185    2.0
186    1.0
187    2.0
188    1.0
189    1.0
190    2.0
191    1.0
Name: frame, Length: 192, dtype: float64

# Save file

In [24]:
choiceFrameAll.to_csv('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/All/PFT_Choice_Pilot_All.csv',index = False)
idFrameAll.to_csv('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots Pet/Data/All/PFT_idRatingsFrame_Pilot_All.csv',index = False)